In [2]:
import requests
import plotly.express as px
import plotly.graph_objects as go
import datetime
import pytz
import folium
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

import pandas as pd
import requests
from pandas import json_normalize

# Les 5 stations les plus proches

In [4]:
# SE CONNECTER A L'API POUR CHARGER NOTRE DATAFRAME

import pandas as pd
import requests
from pandas import json_normalize


service_name = 'stop_areas'
format_type = 'json'
parameters = 'displayCoordXY'
api_key = '15cbfcdf-76bb-4136-980a-6dc1f1d96cd5'

url_avec_parametre = f"https://api.tisseo.fr/v2/{service_name}.{format_type}?displayLines=1&displayCoordXY=1&key={api_key}"
response = requests.get(url_avec_parametre)
data = response.json()

df_stop_areas_coordonnees = pd.DataFrame(data)
df_stop_areas_list_coordonnees = df_stop_areas_coordonnees['stopAreas'].values[0]
df_arrets_coordonnees = pd.DataFrame(df_stop_areas_list_coordonnees)

# MODIFIER NOTRE DATAFRAME

# Suppression des dix premiers caractères de la colonne "id"
df_arrets_coordonnees['id'] = df_arrets_coordonnees['id'].apply(lambda x : x[10:])
# renommage des colonnes du df
df_arrets_coordonnees=df_arrets_coordonnees.rename(columns={"id": "id_stop_area"})
df_arrets_coordonnees=df_arrets_coordonnees.rename(columns={"x": "lon"})
df_arrets_coordonnees=df_arrets_coordonnees.rename(columns={"y": "lat"})

df_arrets_coordonnees['index_arrets'] = df_arrets_coordonnees.index
df_line = df_arrets_coordonnees.explode('line')
df_line = pd.concat([df_line.drop('line', axis=1), df_line['line'].apply(pd.Series)], axis=1)
df_line['index_arrets'] = df_arrets_coordonnees['index_arrets']
df_line_propre = df_line.drop(columns = ['index_arrets','bgXmlColor', 'color', 'fgXmlColor', 'network', 'reservationMandatory', 'id' ])
df_line_propre['transportmode'] = df_line_propre['transportMode'].apply(lambda x: x['name'])

df_line_final = df_line_propre.drop(columns = ['transportMode'])
df_line_final = df_line_final.reset_index(drop=True)
df_line_final.columns = ['arret' if name == 'name' and index == 2 else name for index, name in enumerate(df_line_final.columns)]
df_line_final.rename(columns = {"name":"line"}, inplace=True)

# concatener la latitude avec la longitude
df_line_final['lat&lon'] = df_line_final.apply(lambda x: f"{x['lat']}, {x['lon']}", axis=1)

# Supprimer les lignes en doublons pour avoir une ligne par arrêt : garder les colonnes arrêts, id_stop_area, lat&lon
df_stop_area_unique = df_line_final.drop_duplicates(subset = "id_stop_area")

# Garder uniquement les colonnes souhaitées
df_stop_area_unique = df_stop_area_unique[['cityName', 'id_stop_area', 'arret',  'lat&lon' ]]

/tmp/ipykernel_97/1856130736.py:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
df_stop_area_unique

,cityName,id_stop_area,arret,lat&lon
0,MURET,SA_2127,08 Mai 1945,"43.444718, 1.318037"
1,PLAISANCE-DU-TOUCH,SA_2185,11 Novembre 1918,"43.569672, 1.287371"
2,FONSORBES,SA_652,11 Novembre 1918,"43.535702, 1.229646"
4,COLOMIERS,SA_214,14ème Régiment d'Infanterie,"43.613816, 1.331311"
5,BEAUZELLE,SA_1003,19 mars 1962,"43.661194, 1.384676"
...,...,...,...,...
3098,L'UNION,SA_902,ZI Montredon,"43.638154, 1.500401"
3101,LESPINASSE,SA_14,ZI Petit Paradis,"43.713249, 1.396586"
3102,BALMA,SA_471,ZI Prat Gimont,"43.626924, 1.486025"
3106,AUSSONNE,SA_1488,Zone du Moulin,"43.688312, 1.33712"


In [6]:
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

adresse= '15 Place Arnaud Bernard, 31000, Toulouse'

# Initialisation du géocodeur
geolocator = Nominatim(user_agent="my_app")

location_reference = geolocator.geocode(adresse)

location_reference.latitude, location_reference.longitude

(43.6108946, 1.439271)

In [7]:
# AFFICHER LES STATIONS LES PLUS PROCHES DU PARKING 

from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Adresse de référence
address_reference = input("D'où partez-vous ? \n")

# Liste des coordonnées à comparer
coordinates_to_compare = df_stop_area_unique['lat&lon']

# Initialisation du géocodeur
geolocator = Nominatim(user_agent="my_app")

# Géocodage de l'adresse de référence
location_reference = geolocator.geocode(address_reference)


# Vérification des coordonnées géographiques
if location_reference is not None:

   # Extraction des coordonnées de l'adresse de référence
   coordinates_reference = (location_reference.latitude, location_reference.longitude)

   # Initialisation du dictionnaire pour stocker les distances et les adresses
   distances = {}

   # Parcours des adresses à comparer
   for coordinate_to_compare in coordinates_to_compare:
      distance = geodesic(coordinates_reference, coordinate_to_compare).meters

      # Stockage de la distance et de l'adresse dans le dictionnaire
      distances[coordinate_to_compare] = distance

   # Tri du dictionnaire par distance et récupération des 5 arrêts les plus proches
   closest_stations = sorted(distances, key=distances.get)[:5]

   if closest_stations :

      # Affichage des éléments suivants :

      closest_stops = df_stop_area_unique[df_stop_area_unique['lat&lon'].isin(closest_stations)]['arret'].tolist()
      closest_stops_id = df_stop_area_unique[df_stop_area_unique['lat&lon'].isin(closest_stations)]['id_stop_area'].tolist()
      LatLon_closest_stops = df_stop_area_unique[df_stop_area_unique['lat&lon'].isin(closest_stations)]['lat&lon'].tolist()
      print("Arrêts les plus proches :", closest_stops)
      print("id les plus proches :", closest_stops_id)
      print("lat&lon les plus proches :", LatLon_closest_stops)

      print("Distances :")
      list_distance = []
      for station in closest_stations:
          distance = distances[station]
          print("{:.0f} mètres".format(distance))
          list_distance.append(distances[station])  

   else:
      print("Aucune adresse trouvée parmi la liste")
else:
    print("Adresse de référence introuvable")

# Place Arnaud Bernard, 31000, Toulouse

Arrêts les plus proches : ['A. Bernard', 'Compans-Caffarelli', 'Concorde', 'Embarthe', 'Pont des Minimes']
id les plus proches : ['SA_1637', 'SA_1072', 'SA_724', 'SA_736', 'SA_1724']
lat&lon les plus proches : ['43.6115, 1.438846', '43.610415, 1.435302', '43.610632, 1.442897', '43.608093, 1.438419', '43.615609, 1.437413']
Distances :
71 mètres
303 mètres
312 mètres
315 mètres
542 mètres


In [8]:
df_arrets_plus_proche = pd.concat([pd.Series(closest_stops), pd.Series(closest_stops_id), pd.Series(list_distance), pd.Series(LatLon_closest_stops)], axis=1)
df_arrets_plus_proche = df_arrets_plus_proche.rename(columns = {0 : 'Nom station', 1 : 'ID de la station', 2 : 'Distance en mètres', 3 : 'Coordonnées GPS'})
df_arrets_plus_proche['Distance en mètres'] = df_arrets_plus_proche['Distance en mètres'].astype(int)
df_arrets_plus_proche[['lat', 'lon']] = df_arrets_plus_proche['Coordonnées GPS'].str.split(', ', expand=True).astype(float)

df_arrets_plus_proche

,Nom station,ID de la station,Distance en mètres,Coordonnées GPS,lat,lon
0,A. Bernard,SA_1637,70,"43.6115, 1.438846",43.611500,1.438846
1,Compans-Caffarelli,SA_1072,303,"43.610415, 1.435302",43.610415,1.435302
2,Concorde,SA_724,312,"43.610632, 1.442897",43.610632,1.442897
3,Embarthe,SA_736,315,"43.608093, 1.438419",43.608093,1.438419
4,Pont des Minimes,SA_1724,541,"43.615609, 1.437413",43.615609,1.437413


# Afficher une carte avec les 5 stations les plus proches

## Carte avec Folium

In [15]:
# Créer une carte folium avec les 5 stations de notre DF df_arrets_plus_proche
carte_tisseo = folium.Map(location=[43.608093, 1.438419], zoom_start=15)

for i in range(5):
    coordonnees_lat_arret_tisseo = df_arrets_plus_proche['lat'][i]
    coordonnees_lon_arret_tisseo = df_arrets_plus_proche['lon'][i]
    nom_arret_tisseo = f'ARRET_{i + 1}'
    
    #print(nom_arret_tisseo)
    folium.Marker(location=[coordonnees_lat_arret_tisseo, coordonnees_lon_arret_tisseo], popup="<i>Arret :\n</i>"+df_arrets_plus_proche['Nom station'][i]).add_to(carte_tisseo)

carte_tisseo

## Carte avec Plotly

In [18]:
import plotly.graph_objects as go

# Création de la figure
fig = go.Figure()

# Ajout des marqueurs pour les parkings
for i in range(5):
    coordonnees_lat_arret_tisseo = df_arrets_plus_proche['lat'][i]
    coordonnees_lon_arret_tisseo = df_arrets_plus_proche['lon'][i]
    nom_arret_tisseo = f'ARRET_{i + 1}'

    fig.add_trace(go.Scattermapbox(
        lat=[coordonnees_lat_arret_tisseo],
        lon=[coordonnees_lon_arret_tisseo],
        mode='markers',
        marker=go.scattermapbox.Marker(
            #symbol = 6,
            size=20,
            color='#a5282b',
            opacity=1),
        name=df_arrets_plus_proche['Nom station'][i],
        text = [f" <b>Arrêt:</b> {df_arrets_plus_proche['Nom station'][i]}"],
        hoverinfo = ['text']    
)
    )

# Configuration de la carte
fig.update_layout(
    mapbox_style='open-street-map',
    mapbox_center_lon=df_arrets_plus_proche['lon'][0],
    mapbox_center_lat=df_arrets_plus_proche['lat'][0],
    mapbox=dict(
        zoom=15
        )

)
# Affichage de la carte
fig.show()

# Choix utilisateur : arret

In [16]:
# Créer un menu déroulant sur DASH qui permet à l'utilisateur de selectionner l'arret qu'il souhaite 
menu_deroulant_arrets_tisseo = closest_stops
menu_deroulant_arrets_tisseo

['A. Bernard',
 'Compans-Caffarelli',
 'Concorde',
 'Embarthe',
 'Pont des Minimes']

# Afficher les prochains horaires à l'arrêt selectionné :


In [17]:
# ARRET CHOISI pour l'exemple = Concorde. A REMPLACER DANS L'OUTIL DE VISUALISATION UTILISATEUR 
nom_arret_choisi = 'Concorde'

# Afficher toutes les lignes a l'arrêt indiqué :
df_lignes_arret_choisies = df_line_final[df_line_final['arret'] == nom_arret_choisi]

# Récupérer l'id_stop_area de l'ârret selectionné 
id_stop_area_selectionne = df_lignes_arret_choisies['id_stop_area'].iloc[0]

# Connection à l'API en temps réel
import requests

service_name = 'stops_schedules'
format_type = 'xml'
parameters_stop_area = 'stopAreaId=stop_area:'+id_stop_area_selectionne
api_key = '15cbfcdf-76bb-4136-980a-6dc1f1d96cd5'

url_stop_area_id = f"https://api.tisseo.fr/v2/{service_name}.json?{parameters_stop_area}&key={api_key}"
response = requests.get(url_stop_area_id)
data = response.json()

# Mise en forme d'un dataframe df_schedules_stop_area via les éléments de la colonne stop_area
df_schedules_stop_area = pd.json_normalize(data['departures'], record_path= "departure", meta= 'stopArea')
df_schedules_stop_area = df_schedules_stop_area[['dateTime', 'destination', 'line.name', 'line.network', 'line.shortName', 'stopArea']]
df_schedules_stop_area = pd.concat([df_schedules_stop_area.drop(['stopArea'], axis=1), df_schedules_stop_area['stopArea'].apply(pd.Series)], axis=1)
df_schedules_stop_area = df_schedules_stop_area.drop(['cityId'], axis=1)
df_schedules_stop_area = df_schedules_stop_area.explode('destination')

# Mise en forme d'un dataframe df_destination à partir des éléments de la colonne destination
df_destination = df_schedules_stop_area['destination'].apply(pd.Series)
df_destination = df_destination.rename(columns={"id": "id_stop_area_destination", "name": "name_stop_area_destination", "cityName": "cityName_destination"})

# Concatenation des 2 DF ci-dessus
df_concat = pd.concat([df_schedules_stop_area, df_destination], axis=1)
df_concat = df_concat.drop(['destination', 'line.network', 'id_stop_area_destination', 'name_stop_area_destination'], axis=1)
df_concat = df_concat.rename(columns={"id": "id_stop_area"})
df_concat['id_stop_area'] = df_concat['id_stop_area'].apply(lambda x : x[10:])

# Formatage du DF affiché pour l'utilisateur : 
df_horaires_utilisateur = df_concat.drop(columns=['name', 'cityName', 'id_stop_area'], axis = 1)

df_horaires_utilisateur = df_horaires_utilisateur.rename(columns={
    'dateTime': 'Prochains horaires',
    'line.name': 'Nom de la ligne',
    'line.shortName': 'Numéro de ligne',
    'cityName_destination': 'Ville de destination'
})

# Définition du nouvel ordre des colonnes Réorganisation des colonnes dans le nouvel ordre
nouvel_ordre = ['Prochains horaires', 'Numéro de ligne', 'Nom de la ligne', 'Ville de destination' ]
df_horaires_utilisateur = df_horaires_utilisateur.reindex(columns=nouvel_ordre)


# Voici le dataframe a afficher pour l'utilisateur 
df_horaires_utilisateur

,Prochains horaires,Numéro de ligne,Nom de la ligne,Ville de destination
0,2023-07-20 15:16:02,45,Jeanne d'Arc / Pelletier Purpan,TOULOUSE
1,2023-07-20 15:17:33,L1,Sept Deniers - Salvador Dali / Fonsegrives Ent...,TOULOUSE
2,2023-07-20 15:18:37,14,Basso Cambo / Jean Jaurès,TOULOUSE
3,2023-07-20 15:21:51,29,Grand Rond / Aucamville Collège ou Fonbeauzard...,TOULOUSE
4,2023-07-20 15:21:51,70,Jeanne d'Arc / Aéroconstellation,BEAUZELLE
5,2023-07-20 15:21:54,45,Jeanne d'Arc / Pelletier Purpan,TOULOUSE
6,2023-07-20 15:23:06,14,Basso Cambo / Jean Jaurès,TOULOUSE
7,2023-07-20 15:25:56,L1,Sept Deniers - Salvador Dali / Fonsegrives Ent...,QUINT-FONSEGRIVES
8,2023-07-20 15:34:23,70,Jeanne d'Arc / Aéroconstellation,TOULOUSE
9,2023-07-20 15:37:05,45,Jeanne d'Arc / Pelletier Purpan,TOULOUSE


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7c80b052-7ead-4799-b132-a25d57707958' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>